# Game of 24

In [ ]:
# Setup
import re
import os
import json
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

#############################################
# Just some util and plot styling functions #
#############################################
def get_files_in_folder(folder_path):
    files = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            files.append(folder_path + "/" +file)
    return files

def get_number(string):
    numbers = re.findall(r'\d+', string)
    numbers = list(map(int, numbers))
    return numbers[0] 
    
def get_params(file_path):
    data = {}

    file_name = file_path.split('/')[-1]
    params = file_name.split('_')

    data["set"] = params[0].split("-")[0]
    data["n_agents"] = get_number(params[1])
    data["n_steps"] = get_number(params[2])
    data["k"] = get_number(params[3])
    data["origin_value"] = get_number(params[4])
    data["backtrack"] = float("0." + str(get_number(params[5].split(".")[1])))
    data["resampling"] = params[6].split("-")[0]

    with open(file_path, 'r') as file:
        results = json.load(file)

    data["cost"] = results.pop("Cost")
    models = results.pop("Models")
    info = results.pop("Info")
    success = 0
    for experiment in results.values():
        if {"r":1} in experiment["Verifications"]:
            success +=1
    success_rate  = success/len(results)

    data["success_rate"] = success_rate
    data["file_path"] = file_path
    data["name"] = f"{data['n_agents']}agents\n{data['n_steps']}steps\n{data['k']}k\n{data['backtrack']}b"

    return data

def plot_sucess_rates(df_temp, ax1, ax2, tot_accuracy, tot_cost, sorted_on="NA"):
    # Plot the success rate on the first axes
    df_temp.plot(x="name", y="success_rate", kind="bar", title=f"Success Rate (sorted on {sorted_on})", ax=ax1, alpha=0.2)
    ax1.axhline(y=tot_accuracy, linestyle='--')  # Adding a horizontal line
    ax1.set_ylabel('success rate')
    ax1.grid(True)
    ax1.set_xticklabels([])  # Hide x-axis tick labels

    #Plot the cost on the second axes
    df_temp.plot(x="name", y="total_cost", kind="bar", title=f"Cost (sorted on {sorted_on})", ax=ax2, color="red", alpha=0.2)
    ax2.axhline(y=tot_cost, linestyle='--', color="red")  # Adding a horizontal line
    ax2.set_xlabel('parameters')
    ax2.set_ylabel('cost ($)')

    labels  = df_temp["name"].tolist()
    labels = [label if i%2==0 else "\n\n\n\n"+label for i, label in enumerate(labels)]
    ax2.set_xticklabels(labels, rotation=0) 
    ax2.grid(True)

def plot_per_agents_and_steps(df, tot_accuracy, tot_cost, k, save_to, figsize=(18, 12)):
    n_steps = sorted(df["n_steps"].unique(), reverse=True)
    n_agents = sorted(df["n_agents"].unique(), reverse=True)
    ks = sorted(df["k"].unique(), reverse=False)
    backtracking = df["backtrack"].unique()
    max_accuracy = df["success_rate"].max()
    max_cost = df["total_cost"].max()
    width = 0.4

    df.sort_values(by=["backtrack"], inplace=True)

    fig, axes = plt.subplots(len(n_steps), len(n_agents), figsize=figsize)

    for i, steps in enumerate(n_steps):
            for j, agents in enumerate(n_agents):
                cost_axis = axes[i,j].twinx()
                df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="backtrack", y="success_rate", ax=axes[i,j], title=f"{steps} steps, {agents} agents", kind="bar", alpha=0.2, width=-width, position=-width/2, label="FoA success_rate")
                df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="backtrack", y="total_cost", ax=cost_axis, title=f"{steps} steps, {agents} agents", kind="bar", color="red", alpha=0.2, width=width, position=width/2, label="FoA total_cost")
                axes[i,j].set_ylim(0, max_accuracy*1.1)
                axes[i,j].set_xlim(-0.65, len(backtracking)-0.5)
                cost_axis.set_ylim(0, max_cost*1.1)

                # Adding ToT benchmarks
                axes[i,j].axhline(y=tot_accuracy, color='b', linestyle='--', alpha=0.2, label="ToT success_rate")  # Adding a horizontal line
                cost_axis.axhline(y=tot_cost, color='r', linestyle='--', alpha=0.2, label="ToT total_cost")  # Adding a horizontal line

                # Remove legends
                axes[i,j].legend().remove()
                cost_axis.legend().remove()
                
                if i!=len(n_steps)-1:
                    axes[i,j].set_xticklabels([])
                    axes[i,j].set_xlabel("")
                
                if j==0:
                    axes[i,j].set_ylabel("Success Rate")
                
                if j==len(n_agents)-1:
                    cost_axis.set_ylabel("Cost ($)")

    # Combining legends
    lines, labels = axes[i,j].get_legend_handles_labels()
    lines2, labels2 = cost_axis.get_legend_handles_labels()
    plt.figlegend(lines + lines2, labels + labels2)

    plt.suptitle(f"Success Rate and Cost for different backtrack values (k={k})")

    plt.tight_layout()
    plt.savefig(save_to)
    plt.show()

### ToT Benchmarks

In [ ]:
# Accuracy and Cost from ToT benchmarks
tot_runs = get_files_in_folder("arxiv/benchmarks/gameof24/ToT")
tot_accuracies = []
tot_costs = []

for run_file in tot_runs:
    with open(run_file, 'r') as file:
        run = json.load(file)

    # Compute cost
    cost = run[-1]["usage_so_far"]["cost"]
    tot_costs.append(cost)

    # Compute accuracy
    success = 0
    for puzzle in run:
        if {"r":1} in puzzle["infos"]:
            success +=1
    accuracy = success/len(run)
    tot_accuracies.append(accuracy)

tot_cost = sum(tot_costs)/len(tot_costs)
tot_accuracy = sum(tot_accuracies)/len(tot_accuracies)

print(f"Costs :      {[round(cost,2) for cost in tot_costs]}\t->\tMean: {tot_cost:.3f}")
print(f"Accuracies : {[round(accuracy,2) for accuracy in tot_accuracies]}\t->\tMean: {tot_accuracy:.3f}")

## Finetuned gridsearch

In [ ]:
files = get_files_in_folder("logs_recent/debug/gameof24/gridsearch")

results = [get_params(file) for file in files]
results = sorted(results, key=lambda x: x["success_rate"], reverse=True)

df = pd.DataFrame(results)
df["total_cost"] = df["cost"].apply(lambda x: x["Total cost"]["total_cost"]*2)
df["name"] = df.apply(lambda x: f"{x['n_agents']}agents\n{x['n_steps']}steps\n{x['k']}k\n{x['backtrack']}b", axis=1)

In [ ]:
# prioritizing cost

import matplotlib.pyplot as plt

# Create a figure with two subplots
fig, axes = plt.subplots(2, 2, figsize=(17, 6))

start = 10
end = 30

# Plot sorted with success rate
df_temp = df.copy()
df_temp.sort_values(by=["success_rate", "total_cost"], ascending=[False, True], inplace=True)
df_temp = df_temp.iloc[start:end]
plot_sucess_rates(df_temp, axes[0,0], axes[1,0], tot_accuracy, tot_cost, sorted_on="success rate")

# Plot sorted with cost
df_temp = df.copy()
#df_temp = df_temp[df_temp.success_rate > 0.08]
df_temp.sort_values(by=["total_cost", "success_rate"], ascending=[True, False], inplace=True)
df_temp = df_temp.iloc[start:end]
plot_sucess_rates(df_temp, axes[0,1], axes[1,1], tot_accuracy, tot_cost, sorted_on="cost")

# Adjust layout
plt.tight_layout()
plt.savefig("pics/gameof24/finetuned_gridsearch/best_sr_cost.png", dpi=300)
plt.show()


In [ ]:
n_steps = sorted(df["n_steps"].unique(), reverse=True)
n_agents = sorted(df["n_agents"].unique(), reverse=True)
ks = sorted(df["k"].unique(), reverse=False)
backtracking = df["backtrack"].unique()
max_accuracy = df["success_rate"].max()
max_cost = df["total_cost"].max()
width = 0.4

for k in ks:
    temp = df[df.k==1].copy()
    temp.sort_values(by=["backtrack"], inplace=True)
    plot_per_agents_and_steps(temp, tot_accuracy, tot_cost, k, save_to=f"pics/gameof24/finetuned_gridsearch/gridsearch_{k}k.png", figsize=(16, 10))

## Stress test

Running the same setup multiple times and averaging cost and success rate.

In [ ]:
files = get_files_in_folder("logs_recent/stresstest/gameof24")

results = [get_params(file) for file in files]
results = sorted(results, key=lambda x: x["success_rate"], reverse=True)

df = pd.DataFrame(results)
df["input_tokens"] = df["cost"].apply(lambda x: x["input_tokens"])
df["output_tokens"] = df["cost"].apply(lambda x: x["output_tokens"])
df["total_cost"] = df["cost"].apply(lambda x: x["total_cost"])
df["name"] = df.apply(lambda x: f"{x['n_agents']}agents\n{x['n_steps']}steps\n{x['k']}k\n{x['backtrack']}b", axis=1)

assert (df.groupby(["n_agents", "n_steps", "k", "backtrack"]).count() == 5).any().any()

In [ ]:

fig, axes = plt.subplots(1,1, figsize=(9, 6))
cost_axis = axes.twinx()

success_rate_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])["success_rate"].agg(['mean', 'sem'])
total_cost_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])["total_cost"].agg(['mean', 'sem'])


success_rate_data["mean"].plot(kind="bar", yerr=success_rate_data['sem'], width=-width, position=-width/2, ax=axes, alpha=0.2, label="FoA success_rate")
total_cost_data["mean"].plot(kind="bar",yerr=total_cost_data["sem"], width=width, position=+width/2, ax=cost_axis, color="red", alpha=0.2, label="FoA total_cost")

axes.axhline(y=tot_accuracy, color='b', linestyle='--', alpha=0.2, label="ToT success_rate")  # Adding a horizontal line
cost_axis.axhline(y=tot_cost, color='r', linestyle='--', alpha=0.2, label="ToT total_cost")  # Adding a horizontal line

# No point in plotting CIs for the ToT benchmarks -> ci for success_rate is way too wide
#sr_ci = stats.t.interval(0.95, len(tot_accuracies)-1, loc=np.mean(tot_accuracies), scale=stats.sem(tot_accuracies))
#axes.fill_between(x=range(-1,100) ,y1=sr_ci[0], y2=sr_ci[1], color='blue', alpha=0.1, label = "ToT success rate")
#cost_ci = stats.t.interval(0.95, len(tot_costs)-1, loc=np.mean(tot_costs), scale=stats.sem(tot_costs))
#cost_axis.fill_between(x=range(-1,100) ,y1=cost_ci[0], y2=cost_ci[1], color='red', alpha=0.1, label = "ToT cost")


axes.set_ylabel("Success rate")
cost_axis.set_ylabel("Cost ($)")
axes.set_xlabel(None)

# Combining legends
lines, labels = axes.get_legend_handles_labels()
lines2, labels2 = cost_axis.get_legend_handles_labels()
plt.figlegend(lines + lines2, labels + labels2, loc=(0.5,0.77))

# x-axis tick labels
x_labels  = df.groupby(["n_agents", "n_steps", "k", "backtrack"]).apply(lambda x: x["name"]).drop_duplicates().tolist()
x_labels = [label if i%2==0 else "\n\n\n\n"+label for i, label in enumerate(x_labels)]
axes.set_xticklabels(x_labels, rotation=0) 
axes.grid(True)

# Adjust layout
plt.xlim(-0.75, 11.5)
plt.title("Success rate and cost for chosen configurations")
plt.tight_layout()
plt.savefig("pics/gameof24/stresstest.png", dpi=300)
plt.show()

In [ ]:
#################
# Presentation #
#################

files = get_files_in_folder("logs_old/presentation/gameof24")

results = [get_params(file) for file in files]
results = sorted(results, key=lambda x: x["success_rate"], reverse=True)

df = pd.DataFrame(results)
df["input_tokens"] = df["cost"].apply(lambda x: x["input_tokens"])
df["output_tokens"] = df["cost"].apply(lambda x: x["output_tokens"])
df["total_cost"] = df["cost"].apply(lambda x: x["total_cost"])
df["name"] = df.apply(lambda x: f"{x['n_agents']}agents\n{x['n_steps']}steps\n{x['k']}k\n{x['backtrack']}b", axis=1)

assert (df.groupby(["n_agents", "n_steps", "k", "backtrack"]).count() == 5).any().any()


fig, axes = plt.subplots(1,1, figsize=(9, 6))
cost_axis = axes.twinx()

success_rate_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])["success_rate"].agg(['mean', 'sem'])
total_cost_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])["total_cost"].agg(['mean', 'sem'])


success_rate_data["mean"].plot(kind="bar", yerr=success_rate_data['sem'], width=-width, position=-width/2, ax=axes, alpha=0.2, label="FoA success_rate")
total_cost_data["mean"].plot(kind="bar",yerr=total_cost_data["sem"], width=width, position=+width/2, ax=cost_axis, color="red", alpha=0.2, label="FoA total_cost")

axes.axhline(y=tot_accuracy, color='b', linestyle='--', alpha=0.2, label="ToT success_rate CI")  # Adding a horizontal line
cost_axis.axhline(y=tot_cost, color='r', linestyle='--', alpha=0.2, label="ToT total_cost CI")  # Adding a horizontal line

# No point in plotting CIs for the ToT benchmarks -> ci for success_rate is way too wide
sr_ci = stats.t.interval(0.95, len(tot_accuracies)-1, loc=np.mean(tot_accuracies), scale=stats.sem(tot_accuracies))
cost_ci = stats.t.interval(0.95, len(tot_costs)-1, loc=np.mean(tot_costs), scale=stats.sem(tot_costs))
axes.fill_between(x=range(-1,100) ,y1=sr_ci[0], y2=sr_ci[1], color='blue', alpha=0.1)
cost_axis.fill_between(x=range(-1,100) ,y1=cost_ci[0], y2=cost_ci[1], color='red', alpha=0.1)



axes.set_ylabel("Success rate")
cost_axis.set_ylabel("Cost ($)")
axes.set_xlabel(None)

# Combining legends
lines, labels = axes.get_legend_handles_labels()
lines2, labels2 = cost_axis.get_legend_handles_labels()
plt.figlegend(lines + lines2, labels + labels2, loc=(0.08,0.78), facecolor='white', framealpha=0.9)


# x-axis tick labels
x_labels  = df.groupby(["n_agents", "n_steps", "k", "backtrack"]).apply(lambda x: x["name"]).drop_duplicates().tolist()
x_labels = [label for i, label in enumerate(x_labels)]
axes.set_xticklabels(x_labels, rotation=0) 
axes.grid(True)

# Adjust layout
plt.xlim(-0.75, 4.5)
axes.set_ylim(0, 0.32)
plt.title("Success rate and cost for chosen configurations")
plt.tight_layout()
plt.savefig("pics/gameof24/presentation_CI.png", dpi=300)
plt.show()

# Crosswords

In [5]:
a = {"h1. apple": 2.5, "h2. banana": 1.0, "h3. apple": 0.5, "h4. apple": 0.4}
b = a.copy()
b = {}
a,b

({'h1. apple': 2.5, 'h2. banana': 1.0, 'h3. apple': 0.5, 'h4. apple': 0.4}, {})

In [ ]:
CrosswordsAgent.get_metrics2("logs_recent/debug/crosswords/gridsearch/test-set_6agents_6steps_1k_0origin_0.0backtrack_linear-resampling.json")

In [1]:
import os
import re
import json

def get_files_in_folder(folder_path):
    files = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            files.append(folder_path + "/" +file)
    return files

def get_crossword_metrcis_tot(file_path):

    with open(file_path) as f:
        results = json.load(f)

    cost = results.pop(-1)["cost"]
    best_steps = []
    for game in results:
        step_len = [len(step["actions"]) for step in game]
        if step_len == []:
            # Empty game -> No suggestions at root node
            best_steps.append({"total_step":0, "env_step":0, "actions":[], 'info': {'r_letter': 0, 'r_word': 0},})
            continue
        best_step_index = step_len.index(max(step_len))
        best_step = game[best_step_index]
        best_steps.append(best_step)

    r_letters = [game["info"]["r_letter"] for game in best_steps]
    r_words = [game["info"]["r_word"] for game in best_steps]
    r_game = [1 if game["info"]["r_word"]==1 else 0 for game in best_steps]

    mean_r_letter = sum(r_letters) / len(r_letters)
    mean_r_word = sum(r_words) / len(r_words)
    mean_r_game = sum(r_game) / len(r_game)

    return {"r_letter": mean_r_letter, "r_word": mean_r_word, "r_game": mean_r_game, "cost":cost}

def get_crossword_metrcis_foa(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    info = data.pop("Info")
    cost = info["Cost"]["Total cost"]["total_cost"]

    results = {}
    for puzzle_idx, puzzle in data.items():
        n_agents = len(puzzle) - 2 # -2 for "puzzle" and "Verifications"

        # Get the number of actions each agent performed for the puzzle
        puzzle_actions = []
        for agent in range(n_agents):
            agent_steps = puzzle[f"Agent {agent}"]
            agent_actions = []
            for step_id, step in agent_steps.items():
                actions = step.get("Step", "").split(" -> ")
                n_actions =  len(actions)
                agent_actions.append(n_actions)
            puzzle_actions.append(agent_actions)

        # Get the best state for each agent
        best_states = {}
        for agent in range(n_agents):
            best_states[f"Agent {agent}"] = {}
            best_step_idx = puzzle_actions[agent].index(max(puzzle_actions[agent]))
            best_states[f"Agent {agent}"]["Best step idx"] = best_step_idx
            best_states[f"Agent {agent}"]["Best step n_actions"] = max(puzzle_actions[agent])
            best_states[f"Agent {agent}"]["Best step"] = data[puzzle_idx][f"Agent {agent}"][f"Step {best_step_idx}"]
        
        best_agent = max(best_states, key=lambda x: best_states[x]["Best step n_actions"])
        results[puzzle_idx] = best_states[best_agent]

    r_letters = []
    r_words = []
    r_alls = []
    
    for puzzle_idx, result in results.items():
        r_letters.append(result["Best step"]["metrics"]["r_letter"])
        r_words.append(result["Best step"]["metrics"]["r_word"])
        r_alls.append(result["Best step"]["metrics"]["r_all"])

    r_letters_mean = sum(r_letters) / len(r_letters)
    r_words_mean = sum(r_words) / len(r_words)
    r_alls_mean = sum(r_alls) / len(r_alls)

    return {"r_letter": r_letters_mean, "r_word": r_words_mean, "r_all": r_alls_mean, "total_cost":cost}

def get_params(file_path):
    data = {}

    file_name = file_path.split('/')[-1]
    params = file_name.split('_')

    data["set"] = params[0].split("-")[0]
    data["n_agents"] = get_number(params[1])
    data["n_steps"] = get_number(params[2])
    data["k"] = get_number(params[3])
    data["origin_value"] = get_number(params[4])
    data["backtrack"] = float("0." + str(get_number(params[5].split(".")[1])))
    data["resampling"] = params[6].split("-")[0]

    metrics = get_crossword_metrcis_foa(file_path)

    data.update(metrics)
    data["file_path"] = file_path
    data["name"] = f"{data['n_agents']}agents\n{data['n_steps']}steps\n{data['k']}k\n{data['backtrack']}b"

    return data

def plot_metric(df_temp, metric, tot_metrics, ax1, ax2, tot_accuracy, tot_cost, sorted_on="NA"):
    # Plot the success rate on the first axes
    df_temp.plot(x="name", y=metric, kind="bar", title=f"{metric} (sorted on {sorted_on})", ax=ax1, alpha=0.2)
    ax1.axhline(y=tot_metrics[metric], linestyle='--')  # Adding a horizontal line
    ax1.set_ylabel(metric)
    ax1.grid(True)
    ax1.set_xticklabels([])  # Hide x-axis tick labels

    #Plot the cost on the second axes
    df_temp.plot(x="name", y="total_cost", kind="bar", title=f"Cost (sorted on {sorted_on})", ax=ax2, color="red", alpha=0.2)
    ax2.axhline(y=tot_metrics["total_cost"], linestyle='--', color="red")  # Adding a horizontal line
    ax2.set_xlabel('parameters')
    ax2.set_ylabel('cost ($)')

    labels  = df_temp["name"].tolist()
    labels = [label if i%2==0 else "\n\n\n\n"+label for i, label in enumerate(labels)]
    ax2.set_xticklabels(labels, rotation=0) 
    ax2.grid(True)

def plot_per_agents_and_steps(df, metric, tot_metric, tot_cost, k, save_to, figsize=(18, 12)):
    n_steps = sorted(df["n_steps"].unique(), reverse=True)
    n_agents = sorted(df["n_agents"].unique(), reverse=True)
    ks = sorted(df["k"].unique(), reverse=False)
    backtracking = df["backtrack"].unique()
    max_metric = df[metric].max()
    max_cost = df["total_cost"].max()
    width = 0.4

    df.sort_values(by=["backtrack"], inplace=True)

    fig, axes = plt.subplots(len(n_steps), len(n_agents), figsize=figsize)

    for i, steps in enumerate(n_steps):
            for j, agents in enumerate(n_agents):
                cost_axis = axes[i,j].twinx()
                df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="backtrack", y=metric, ax=axes[i,j], title=f"{steps} steps, {agents} agents", kind="bar", alpha=0.2, width=-width, position=-width/2, label=f"FoA {metric}")
                df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="backtrack", y="total_cost", ax=cost_axis, title=f"{steps} steps, {agents} agents", kind="bar", color="red", alpha=0.2, width=width, position=width/2, label="FoA total_cost")
                axes[i,j].set_ylim(0, max_accuracy*1.1)
                axes[i,j].set_xlim(-0.65, len(backtracking)-0.5)
                cost_axis.set_ylim(0, max_cost*1.1)

                # Adding ToT benchmarks
                axes[i,j].axhline(y=tot_metric, color='b', linestyle='--', alpha=0.2, label=f"ToT {metric}")  # Adding a horizontal line
                cost_axis.axhline(y=tot_cost, color='r', linestyle='--', alpha=0.2, label="ToT total_cost")   # Adding a horizontal line

                # Remove legends
                axes[i,j].legend().remove()
                cost_axis.legend().remove()
                
                if i!=len(n_steps)-1:
                    axes[i,j].set_xticklabels([])
                    axes[i,j].set_xlabel("")
                
                if j==0:
                    axes[i,j].set_ylabel(f"{metric}")
                
                if j==len(n_agents)-1:
                    cost_axis.set_ylabel("Cost ($)")

    # Combining legends
    lines, labels = axes[i,j].get_legend_handles_labels()
    lines2, labels2 = cost_axis.get_legend_handles_labels()
    plt.figlegend(lines + lines2, labels + labels2)

    plt.suptitle(f"{metric.capitalize()} and Cost for different backtrack values (k={k})")

    plt.tight_layout()
    plt.savefig(save_to)
    plt.show()

def plot_cross(df, metric, tot_metric, tot_cost, backtrack, save_to, figsize=(18, 12)):
    n_steps = sorted(df["n_steps"].unique(), reverse=True)
    n_agents = sorted(df["n_agents"].unique(), reverse=True)
    ks = df["k"].unique()
    max_metric = df[metric].max()
    max_cost = df["total_cost"].max()
    width = 0.4

    print(f"n_steps: {n_steps}")
    print(f"n_agents: {n_agents}")

    df.sort_values(by=["backtrack"], inplace=True)

    fig, axes = plt.subplots(len(n_steps), len(n_agents), figsize=figsize)

    for i, steps in enumerate(n_steps):
        for j, agents in enumerate(n_agents):
            cost_axis = axes[i,j].twinx()
            df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="k", y=metric, ax=axes[i,j], title=f"{steps} steps, {agents} agents", kind="bar", alpha=0.2, width=-width, position=-width/2, label=f"FoA {metric}")
            df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="k", y="total_cost", ax=cost_axis, title=f"{steps} steps, {agents} agents", kind="bar", color="red", alpha=0.2, width=width, position=width/2, label="FoA total_cost")
            axes[i,j].set_ylim(0, max_metric*1.1)
            axes[i,j].set_xlim(-0.65, len(ks)-0.5)
            cost_axis.set_ylim(0, max_cost*1.1)

            # Adding ToT benchmarks
            axes[i,j].axhline(y=tot_metric, color='b', linestyle='--', alpha=0.2, label=f"ToT {metric}")  # Adding a horizontal line
            cost_axis.axhline(y=tot_cost, color='r', linestyle='--', alpha=0.2, label="ToT total_cost")   # Adding a horizontal line

            # Remove legends
            axes[i,j].legend().remove()
            cost_axis.legend().remove()
            
            if i!=len(n_steps)-1:
                axes[i,j].set_xticklabels([])
                axes[i,j].set_xlabel("")
            
            if j==0:
                axes[i,j].set_ylabel(f"{metric}")
            
            if j==len(n_agents)-1:
                cost_axis.set_ylabel("Cost ($)")

    # Combining legends
    lines, labels = axes[i,j].get_legend_handles_labels()
    lines2, labels2 = cost_axis.get_legend_handles_labels()
    plt.figlegend(lines + lines2, labels + labels2)

    plt.suptitle(f"{metric.capitalize()} and Cost for different k values (backtrack={backtrack})")

    plt.tight_layout()
    plt.savefig(save_to)
    plt.show()

def get_number(string):
    numbers = re.findall(r'\d+', string)
    numbers = list(map(int, numbers))
    return numbers[0] 

def plot_cross(df, metric, tot_metric, tot_cost, backtrack, save_to, figsize=(18, 12)):
    n_steps = sorted(df["n_steps"].unique(), reverse=True)
    n_agents = sorted(df["n_agents"].unique(), reverse=True)
    ks = df["k"].unique()
    max_metric = df[metric].max()
    max_cost = df["total_cost"].max()
    width = 0.4

    fig, axes = plt.subplots(len(n_steps), len(n_agents), figsize=figsize)

    for i, steps in enumerate(n_steps):
        for j, agents in enumerate(n_agents):
            cost_axis = axes[i,j].twinx()
            df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="k", y=metric, ax=axes[i,j], title=f"{steps} steps, {agents} agents", kind="bar", alpha=0.2, width=-width, position=-width/2, label=f"FoA {metric}")
            df[(df.n_steps==steps) & (df.n_agents==agents)].plot(x="k", y="total_cost", ax=cost_axis, title=f"{steps} steps, {agents} agents", kind="bar", color="red", alpha=0.2, width=width, position=width/2, label="FoA total_cost")
            axes[i,j].set_ylim(0, max_metric*1.1)
            axes[i,j].set_xlim(-0.65, len(ks)-0.5)
            cost_axis.set_ylim(0, max_cost*1.1)

            # Adding ToT benchmarks
            axes[i,j].axhline(y=tot_metric, color='b', linestyle='--', alpha=0.2, label=f"ToT {metric}")  # Adding a horizontal line
            cost_axis.axhline(y=tot_cost, color='r', linestyle='--', alpha=0.2, label="ToT total_cost")   # Adding a horizontal line

            # Remove legends
            axes[i,j].legend().remove()
            cost_axis.legend().remove()
            
            if i!=len(n_steps)-1:
                axes[i,j].set_xticklabels([])
                axes[i,j].set_xlabel("")
            
            if j==0:
                axes[i,j].set_ylabel(f"{metric}")
            
            if j==len(n_agents)-1:
                cost_axis.set_ylabel("Cost ($)")

    # Combining legends
    lines, labels = axes[i,j].get_legend_handles_labels()
    lines2, labels2 = cost_axis.get_legend_handles_labels()
    plt.figlegend(lines + lines2, labels + labels2)

    plt.suptitle(f"{metric.capitalize()} and Cost for different k values (backtrack={backtrack})")

    plt.tight_layout()
    plt.savefig(save_to)
    plt.show()

### ToT Benchmarks

In [ ]:
benchmark_logs = get_files_in_folder("arxiv/benchmarks/crosswords/ToT")

tot_metrics = {"r_letter_all": [], "r_word_all": [], "r_game_all": [], "total_cost_all": []}

for log in benchmark_logs:
    result = get_crossword_metrcis_tot(log)
    tot_metrics["r_letter_all"].append(result["r_letter"])
    tot_metrics["r_word_all"].append(result["r_word"])
    tot_metrics["r_game_all"].append(result["r_game"])
    tot_metrics["total_cost_all"].append(result["cost"])

tot_metrics.update({key[:-4]: sum(value)/len(value) for key, value in tot_metrics.items()})
tot_metrics

## Initial Gridsearch

In [ ]:
import pandas as pd
files = get_files_in_folder("logs_recent/debug/crosswords/gridsearch")

results = [get_params(file) for file in files]
results = sorted(results, key=lambda x: x["r_letter"], reverse=True)

df = pd.DataFrame(results)
df.head()

In [ ]:
# prioritizing cost

import matplotlib.pyplot as plt

# Create a figure with two subplots
fig, axes = plt.subplots(2, 2, figsize=(17, 6))

metric = "r_letter"
start=0
end=20

# Plot sorted with success rate
df_temp = df.copy()
df_temp = df_temp[df.total_cost<=tot_metrics["total_cost"]]
df_temp.sort_values(by=[metric, "total_cost"], ascending=[False, True], inplace=True)
df_temp = df_temp.head(20)
plot_metric(df_temp, metric, tot_metrics, axes[0,0], axes[1,0], tot_metrics[metric], tot_metrics["total_cost"], sorted_on=metric)

# Plot sorted with cost
df_temp = df.copy()
df_temp = df_temp[df_temp[metric] > tot_metrics[metric]*0.6]
df_temp.sort_values(by=["total_cost", metric], ascending=[True, False], inplace=True)
df_temp = df_temp.iloc[start:end]
plot_metric(df_temp, metric, tot_metrics, axes[0,1], axes[1,1], tot_metrics[metric], tot_metrics["total_cost"], sorted_on="cost")

# Adjust layout
plt.tight_layout()
plt.savefig("pics/gameof24/initial_gridsearch/best_sr_cost.png", dpi=300)
plt.show()

### Aggregated results for k

In [ ]:
metric="r_letter"
for k in df.k.unique():
    temp = df[df.backtrack==k].copy()
    temp.sort_values(by=["backtrack"], inplace=True)
    plot_cross(temp, metric, tot_metrics[metric], tot_metrics["total_cost"], k, save_to=f"pics/crosswords/initial_gridsearch/gridsearch_{metric}_{k}backtrack.png", figsize=(14, 8))

In [ ]:
metric="r_word"
for backtrack in df.backtrack.unique():
    temp = df[df.backtrack==backtrack].copy()
    temp.sort_values(by=["k"], inplace=True)
    plot_cross(temp, metric, tot_metrics[metric], tot_metrics["total_cost"], backtrack, save_to=f"pics/crosswords/initial_gridsearch/gridsearch_{metric}_{backtrack}backtrack.png", figsize=(14, 8))

### Strees test

In [ ]:
files = get_files_in_folder("logs_recent/stresstest/crosswords")

results = [get_params(file) for file in files]
results = sorted(results, key=lambda x: x["r_letter"], reverse=True)



df = pd.DataFrame(results)

#assert (df.groupby(["n_agents", "n_steps", "k", "backtrack"]).count() == 4).any().any()
df.head()

In [ ]:
metric = "r_letter"

fig, axes = plt.subplots(1,1, figsize=(9, 6))
cost_axis = axes.twinx()

success_rate_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])[metric].agg(['mean', 'sem'])
total_cost_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])["total_cost"].agg(['mean', 'sem'])


success_rate_data["mean"].plot(kind="bar", yerr=success_rate_data['sem'], width=-width, position=-width/2, ax=axes, alpha=0.2, label=f"FoA {metric}")
total_cost_data["mean"].plot(kind="bar",yerr=total_cost_data["sem"], width=width, position=+width/2, ax=cost_axis, color="red", alpha=0.2, label="FoA total_cost")

axes.axhline(y=tot_metrics[metric], color='b', linestyle='--', alpha=0.2, label=f"ToT {metric}")  # Adding a horizontal line
cost_axis.axhline(y=tot_metrics["total_cost"], color='r', linestyle='--', alpha=0.2, label="ToT total_cost")  # Adding a horizontal line

# No point in plotting CIs for the ToT benchmarks -> ci for success_rate is way too wide
#sr_ci = stats.t.interval(0.95, len(tot_metrics[metric+"_all"])-1, loc=np.mean(tot_metrics[metric+"_all"]), scale=stats.sem(tot_metrics[metric+"_all"]))
#axes.fill_between(x=range(-1,100) ,y1=sr_ci[0], y2=sr_ci[1], color='blue', alpha=0.1, label = f"CI: ToT {metric}")
#cost_ci = stats.t.interval(0.95, len(tot_metrics["total_cost_all"])-1, loc=np.mean(tot_metrics["total_cost_all"]), scale=stats.sem(tot_metrics["total_cost_all"]))
#cost_axis.fill_between(x=range(-1,100) ,y1=cost_ci[0], y2=cost_ci[1], color='red', alpha=0.1, label = "CI: ToT cost")


axes.set_ylabel(f"{metric}")
cost_axis.set_ylabel("Cost ($)")
axes.set_xlabel(None)

# Combining legends
lines, labels = axes.get_legend_handles_labels()
lines2, labels2 = cost_axis.get_legend_handles_labels()
plt.figlegend(lines + lines2, labels + labels2, loc=(0.5,0.77))

# x-axis tick labels
x_labels  = df.groupby(["n_agents", "n_steps", "k", "backtrack"]).apply(lambda x: x["name"]).drop_duplicates().tolist()
x_labels = [label if i%2==0 else "\n\n\n\n"+label for i, label in enumerate(x_labels)]
axes.set_xticklabels(x_labels, rotation=0) 
axes.grid(True)

# Adjust layout
plt.xlim(-0.75, 11.5)
plt.title(f"{metric.capitalize()} and cost for chosen configurations")
plt.tight_layout()
plt.savefig("pics/crosswords/stresstest.png", dpi=300)
plt.show()

In [ ]:
################
# Presentation #
################

files = get_files_in_folder("logs_old/presentation/crosswords")

results = [get_params(file) for file in files]
results = sorted(results, key=lambda x: x["r_letter"], reverse=True)



df = pd.DataFrame(results)
df["name"]=df.name.str.replace("\n\n\n", "")

#assert (df.groupby(["n_agents", "n_steps", "k", "backtrack"]).count() == 4).any().any()
df.head()

metric = "r_letter"

fig, axes = plt.subplots(1,1, figsize=(9, 6))
cost_axis = axes.twinx()

success_rate_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])[metric].agg(['mean', 'sem'])
total_cost_data = df.groupby(["n_agents", "n_steps", "k", "backtrack"])["total_cost"].agg(['mean', 'sem'])


success_rate_data["mean"].plot(kind="bar", yerr=success_rate_data['sem'], width=-width, position=-width/2, ax=axes, alpha=0.2, label=f"FoA {metric}")
total_cost_data["mean"].plot(kind="bar",yerr=total_cost_data["sem"], width=width, position=+width/2, ax=cost_axis, color="red", alpha=0.2, label="FoA total_cost")

axes.axhline(y=tot_metrics[metric], color='b', linestyle='--', alpha=0.2, label=f"ToT {metric}")  # Adding a horizontal line
cost_axis.axhline(y=tot_metrics["total_cost"], color='r', linestyle='--', alpha=0.2, label="ToT total_cost")  # Adding a horizontal line

# No point in plotting CIs for the ToT benchmarks -> ci for success_rate is way too wide
sr_ci = stats.t.interval(0.95, len(tot_metrics[metric+"_all"])-1, loc=np.mean(tot_metrics[metric+"_all"]), scale=stats.sem(tot_metrics[metric+"_all"]))
axes.fill_between(x=range(-1,100) ,y1=sr_ci[0], y2=sr_ci[1], color='blue', alpha=0.1)
cost_ci = stats.t.interval(0.95, len(tot_metrics["total_cost_all"])-1, loc=np.mean(tot_metrics["total_cost_all"]), scale=stats.sem(tot_metrics["total_cost_all"]))
cost_axis.fill_between(x=range(-1,100) ,y1=cost_ci[0], y2=cost_ci[1], color='red', alpha=0.1)


axes.set_ylabel(f"{metric}")
cost_axis.set_ylabel("Cost ($)")
axes.set_xlabel(None)

# Combining legends
lines, labels = axes.get_legend_handles_labels()
lines2, labels2 = cost_axis.get_legend_handles_labels()
plt.figlegend(lines + lines2, labels + labels2, loc=(0.08,0.79), facecolor='white', framealpha=0.9)

# x-axis tick labels
x_labels  = df.groupby(["n_agents", "n_steps", "k", "backtrack"]).apply(lambda x: x["name"]).drop_duplicates().tolist()
x_labels = [label for i, label in enumerate(x_labels)]
axes.set_xticklabels(x_labels, rotation=0) 
axes.grid(True)

# Adjust layout
plt.xlim(-0.75, 2.5)
plt.title(f"{metric.capitalize()} and cost for chosen configurations")
plt.tight_layout()
plt.savefig("pics/crosswords/presentation_CI.png", dpi=300)
plt.show()